In [1]:
import sys
from os.path import expanduser
home = expanduser("~")
sys.path.insert(2, f"{home}/catkin_ws/src/neurocontroller_database/src")
from load_bci_results import get_data_experiments
from utilities import load_json, color_axes, save_json, has_attr, root_dir, get_users, append_data, show_pdf

import numpy as np
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import tensorflow as tf
from scipy import stats
from scipy.stats import wilcoxon
import os

list_users = None
# list_users = ["test"]

dir_out_r = "./results_SAM"
if not os.path.isdir(dir_out_r):
    os.makedirs(dir_out_r)

In [2]:
get_users()

['test_pref_old',
 'test_final',
 'user1',
 'user_test',
 'test_pref_new',
 'compare_pref_model']

+++++++++++++++++++++++
Preferencias por frente
+++++++++++++++++++++++

In [3]:
data_front = get_data_experiments("front", is_obj_space=True,)# users=["user_test"])
data_front[:3]

-- test_pref_old
ERROR: front optimized-10: No se pueden obtener los valores de los objetivos time is not initialized. Have you called init_node()? 
ERROR: front optimized-19: No se pueden obtener los valores de los objetivos time is not initialized. Have you called init_node()? 
ERROR: front optimized-24: No se pueden obtener los valores de los objetivos time is not initialized. Have you called init_node()? 
ERROR: front optimized-29: No se pueden obtener los valores de los objetivos time is not initialized. Have you called init_node()? 
ERROR: front optimized-34: No se pueden obtener los valores de los objetivos time is not initialized. Have you called init_node()? 
ERROR: front optimized-10: No se pueden obtener los valores de los objetivos time is not initialized. Have you called init_node()? 
ERROR: front optimized-19: No se pueden obtener los valores de los objetivos time is not initialized. Have you called init_node()? 
ERROR: front optimized-24: No se pueden obtener los valores

,id pareto,iteración,usuario,type experiment,id experiment,num ind SAM,PBI SAM,valence SAM,arousal SAM
0,optimized-10,1,test_pref_old,traditional,0,3,0.448474,0.0,0.0
1,optimized-19,2,test_pref_old,traditional,0,10,0.448474,0.0,0.0
2,optimized-24,3,test_pref_old,traditional,0,14,0.448474,0.0,0.0


PBI

In [4]:
fig = px.line(data_front, x="iteración", y="PBI SAM", color="usuario")
# fig.update_layout(showlegend=False)
# fig.update_layout(yaxis=dict(range=[0.4,0.8]))
# fig.write_image(f"{dir_out_r}/pbi_max_by_front SAM.pdf")
fig.show()

fig = px.box(data_front, y="PBI SAM", x="usuario", color="usuario", points="all")
# fig.update_layout(showlegend=False)
fig.update_traces(boxmean="sd")
# fig.write_image(f"{dir_out_r}/box_plot_pbi.pdf")
fig.show()

Gráficas de error de los componentes emocionales (valence, arousal)

In [5]:
fig = px.box(data_front, x="usuario", y="valence SAM", color="usuario", points="all")
# fig.update_layout(yaxis=dict(range=[0,1]))
fig.update_traces(boxmean="sd")
fig.show()

fig = px.box(data_front, x="usuario", y="arousal SAM", color="usuario", points="all")
# fig.update_layout(yaxis=dict(range=[0,1]))
fig.update_traces(boxmean="sd")
fig.show()

Preferencias emocionales por frente

In [6]:

fig = px.line(data_front, x="valence SAM", y="arousal SAM", color="usuario", text="iteración")
# muestra los ejes y el espacio de Russell
fig.add_shape(type="line", x0=0.5, y0=0, x1=0.5,y1=1, xref='x', yref='y', line=dict(color=color_axes,))
fig.add_shape(type="line", x0=0, y0=0.5, x1=1,y1=0.5, xref='x', yref='y', line=dict(color=color_axes,))
fig.add_shape(type="circle", x0=0, y0=0, x1=1, y1=1, line=dict(color=color_axes),)
fig.update_yaxes(scaleanchor="x", scaleratio=1,)
fig.update_layout(xaxis=dict(range=[0, 1], autorange=False), yaxis=dict(range=[0, 1], autorange=False))
# fig.update_layout(showlegend=showlegend, font=dict( size=size_num ))
# fig.update_layout(yaxis = dict(tickfont = dict(size=16), titlefont=dict(size=size_label_axe)), xaxis = dict(tickfont = dict(size=16), titlefont=dict(size=size_label_axe)))
fig.update_layout(margin=dict(l=1, r=1, t=1, b=1))
# fig.write_image(f"{dir_out_r}/pref_emo_model.pdf")
fig.show()


In [7]:
def angle_two_vectors(v1,v2,degree=True):
    v = np.clip(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)), -1, 1)
    angle = np.arccos(v)
    if degree:
        return np.degrees(angle)
    else:
        return angle


def get_deviation(emo_values, vector_ref, degree=True):
    size_emo = emo_values.shape[0]
    if size_emo < 2:
        print(f"Emociones insuficientes, solo hay {size_emo}")
        return

    emo_preds1 = emo_values
    # if size_emo == 2:
    #     emo_preds2 = emo_values[-1]
    # else:
    emo_preds2 = np.vstack((emo_preds1[1:], emo_preds1[-1]))
    
    # vector director
    vd_pred = emo_preds2 - emo_preds1
    vd_ref_pred = np.subtract(vector_ref, emo_preds1)
    
    # grados
    angle_pred = []
    for i in range(vd_ref_pred.shape[0]-1):
        angle_pred.append(angle_two_vectors(vd_pred[i], vd_ref_pred[i],degree))
    
    # limpieza de datos
    angle_pred = np.nan_to_num(angle_pred)
    angle_pred = np.concatenate((angle_pred, [0]))

    
    return angle_pred

def get_all_deviations(data, vector_ref, degree=False, plot_bar=True, save_bar=True):
    # por usuario
    data_bar = {}
    vector_ref = np.array(vector_ref)
    angle_sam_all = []

    users_l = np.unique(data["usuario"])

    for user_name in users_l:
        print(user_name)
        # datos del usuario
        data_user = data.loc[(data["usuario"] == user_name) & (data["type experiment"] != "traditional")]
        data_user = data_user.sort_values("iteración")

        # angulos de desviación
        angle_sam = get_deviation(data_user[["valence SAM", "arousal SAM"]].to_numpy(), vector_ref, degree=degree)
        if angle_sam is None:
            continue

        append_data(data_bar, "usuario", user_name)
        append_data(data_bar, "ángulo", np.sum(angle_sam))
        append_data(data_bar, "media", np.mean(angle_sam))
        append_data(data_bar, "std", np.std(angle_sam))
        append_data(data_bar, "emoción", "SAM")

        # registro de las desviaciones
        angle_sam_all = np.concatenate((angle_sam_all, angle_sam))

        # break

    pd_bar = pd.DataFrame(data_bar)

    if save_bar:
        for i,user_name in enumerate(users_l):
            pd_bar.replace(to_replace=user_name, value=f"sujeto {i+1}", inplace=True)

    fig = px.bar(pd_bar, x="usuario", y="ángulo", color="emoción", barmode='group')
    fig.update_xaxes(title="")
    if save_bar: fig.write_image(f"{dir_out_r}/desviation_{vector_ref}.pdf")
    if plot_bar: fig.show()


    rest = {}
    rest["sum SAM"] = np.sum(angle_sam_all)
    rest["mean SAM"] = np.mean(angle_sam_all)
    rest["std SAM"] = np.std(angle_sam_all)
    
    return rest
    


def plot_scatter(data, refs, key, is_plot=False):
    f = px.scatter(pd.DataFrame({"x":refs[:,0], "y":refs[:,1], f"{key}":data[key]}), x="x", y="y", color=f"{key}")
    f.add_shape(type="line", x0=0.5, y0=0, x1=0.5,y1=1, xref='x', yref='y', line=dict(color=color_axes,))
    f.add_shape(type="line", x0=0, y0=0.5, x1=1,y1=0.5, xref='x', yref='y', line=dict(color=color_axes,))
    f.add_shape(type="circle", x0=0, y0=0, x1=1, y1=1, line=dict(color=color_axes),)
    f.update_yaxes(scaleanchor="x", scaleratio=1,)
    f.update_layout(showlegend=True)
    f.write_image(f"{dir_out_r}/affinity_{key}.pdf")
    if is_plot: f.show()


def plot_desviation_map(num_samples):
    dir_des = f"{dir_out_r}/desviation_data.txt"
    
    if not os.path.isfile(dir_des):
        print("calculando desviaciones")
        ref_points =  np.random.random((num_samples,2))
        data_pref = {}

        # valores totales de la desviación
        for ref_p in ref_points:
            result = get_all_deviations(data_front, ref_p, plot_bar=False, save_bar=False)
            for k in result:
                append_data(data_pref, k, result[k])
        
        save_json({"desviation": data_pref, "points":ref_points.tolist()}, dir_des)
    else:
        print("cargando desviaciones")
        desvi = load_json(dir_des)
        data_pref = desvi["desviation"]
        ref_points = np.array(desvi["points"])
        desvi = None

    # mapas desviacionales
    metrics = ["cos sim", "mse", "mae"]
    for k in data_pref:
        is_plot = False
        if k in metrics: is_plot = True
        plot_scatter(data_pref, ref_points, k, is_plot=is_plot)

    print("----------------------------------")
    
    # mapas diferenciales
    keys = ["sum", "mean", "std"]
    for k in keys:
        # diferencia
        dif = np.abs(np.subtract(data_pref[f"{k} SAM"], data_pref[k]))
        
        # mapa desviacional
        show_pdf(f"{dir_out_r}/affinity_{k}.pdf")
        show_pdf(f"{dir_out_r}/affinity_{k} SAM.pdf")

        f = px.scatter(pd.DataFrame({"x":ref_points[:,0], "y":ref_points[:,1], f"dif {k}":dif}), x="x", y="y", color=f"dif {k}")
        f.add_shape(type="line", x0=0.5, y0=0, x1=0.5,y1=1, xref='x', yref='y', line=dict(color=color_axes,))
        f.add_shape(type="line", x0=0, y0=0.5, x1=1,y1=0.5, xref='x', yref='y', line=dict(color=color_axes,))
        f.add_shape(type="circle", x0=0, y0=0, x1=1, y1=1, line=dict(color=color_axes),)
        f.update_yaxes(scaleanchor="x", scaleratio=1,)
        f.update_layout(showlegend=True)
        f.write_image(f"{dir_out_r}/affinity_diff_{k}.pdf")
        f.show()
    

In [8]:
get_all_deviations(data_front, [1,1], save_bar=True)

compare_pref_model
Emociones insuficientes, solo hay 0
test_pref_new
Emociones insuficientes, solo hay 0
test_pref_old
Emociones insuficientes, solo hay 0
user1
user_test


{'sum SAM': 11.71045404442117,
 'mean SAM': 1.4638067555526462,
 'std SAM': 1.0117245472134273}

In [9]:
# show_pdf(f"{dir_out_r}/desviation_[1 1].pdf")
# show_pdf(f"{dir_out_r}/desviation_[1 0].pdf")

++++++++++++++++++++++
Todos los experimentos
++++++++++++++++++++++

In [10]:
data_exp = get_data_experiments("individuo", is_obj_space=True,do_pbi=True,)# users=["nuevo"])
data_exp[:3]

-- test_pref_old
ERROR: No se pueden obtener los valores de los objetivos: time is not initialized. Have you called init_node()? 
ERROR: No se pueden obtener los valores de los objetivos: time is not initialized. Have you called init_node()? 
ERROR: No se pueden obtener los valores de los objetivos: time is not initialized. Have you called init_node()? 
ERROR: No se pueden obtener los valores de los objetivos: time is not initialized. Have you called init_node()? 
ERROR: No se pueden obtener los valores de los objetivos: time is not initialized. Have you called init_node()? 
ERROR: No se pueden obtener los valores de los objetivos: time is not initialized. Have you called init_node()? 
ERROR: No se pueden obtener los valores de los objetivos: time is not initialized. Have you called init_node()? 
ERROR: No se pueden obtener los valores de los objetivos: time is not initialized. Have you called init_node()? 
ERROR: No se pueden obtener los valores de los objetivos: time is not initializ

,id pareto,num ind,num exp,usuario,type experiment,id experiment,PBI SAM,valence SAM,arousal SAM
0,optimized-10,3.0,1.0,test_pref_old,traditional,0,0.448474,0.0,0.0
1,optimized-19,10.0,2.0,test_pref_old,traditional,0,0.448474,0.0,0.0
2,optimized-24,14.0,3.0,test_pref_old,traditional,0,0.448474,0.0,0.0


PBI

In [11]:
# PBI por experimento
fig = px.line(data_exp, x="num exp", y="PBI SAM", color="usuario")
# fig.update_layout(showlegend=False)
# fig.write_image(f"{dir_out_r}/pbi_all_ind.pdf")
fig.show()

fig = px.box(data_exp, y="PBI SAM", x="usuario", color="usuario", points="all")
# fig.update_layout(showlegend=False)
fig.update_traces(boxmean="sd")
# fig.write_image(f"{dir_out_r}/box_plot_pbi.pdf")
fig.show()

Gráficas de error de los componentes emocionales (valence, arousal)

In [12]:
fig = px.box(data_exp, x="usuario", y="valence SAM", color="usuario", points="all")
# fig.update_layout(yaxis=dict(range=[0,1]))
fig.update_traces(boxmean="sd")
fig.show()

fig = px.box(data_exp, x="usuario", y="arousal SAM", color="usuario", points="all")
# fig.update_layout(yaxis=dict(range=[0,1]))
fig.update_traces(boxmean="sd")
fig.show()

Histogramas de la valencia y el arousal

In [13]:
group_labels = ['valence', "arousal"]
fig = ff.create_distplot(data_exp[["valence SAM", "arousal SAM"]].to_numpy().transpose(1,0), group_labels, bin_size=0.05)
fig.show()


histo = {}
append_data(histo, "valor", data_exp["valence SAM"], "concatenate")
append_data(histo, "emoción", ["valence"]*data_exp["valence SAM"].shape[0], "concatenate")

append_data(histo, "valor", data_exp["arousal SAM"], "concatenate")
append_data(histo, "emoción", ["arousal"]*data_exp["valence SAM"].shape[0], "concatenate")

fig = px.histogram(pd.DataFrame(histo), x="valor", color="emoción",  marginal="rug",)
fig.show()

fig = px.box(pd.DataFrame(histo), y="valor", color="emoción")
fig.update_traces(boxmean="sd")
fig.show()

Emociones de TODOS los neurocontroladores por usuario

In [14]:


def get_range(low, high, percent=0.1):
    size_r = abs(high - low)
    s_portion = size_r * percent

    if s_portion == 0:
        s_portion = 0.01

    return low - s_portion, high + s_portion, s_portion


def plot_all_roadmap(data ,key_x, key_y, scale=0.01, users=None):

    
    users_l = np.unique(data["usuario"])

    # data_by_front = {}
    for user_name in users_l:
        
        data_user = data.loc[(data["usuario"] == user_name) & (data["type experiment"] != "traditional")]
        
        lx, hx, p1 = get_range(data_user[key_x].min(), data_user[key_x].max(), scale)
        ly, hy, p2 = get_range(data_user[key_y].min(), data_user[key_y].max(), scale)
        

        fig = go.Figure()
        # frentes
        unique_fronts = np.unique(data_user["id pareto"])
        unique_fronts = [int(id.split("-")[1]) for id in unique_fronts]
        unique_fronts.sort()
        unique_fronts = [f"optimized-{n}" for n in unique_fronts]

        # puntos de experimentos por frente
        for id_pareto in unique_fronts:
            data_by_front = data_user.loc[data_user["id pareto"] == id_pareto]
            # text = np.arange(len(data_by_front[key_x]), dtype=int).astype(str)
            text = data_by_front["num exp"].to_numpy().astype(int).astype(str)
            fig.add_traces(go.Scatter(x=data_by_front[key_x], y=data_by_front[key_y], mode='markers+lines+text', name=id_pareto, text=text))
            
        data_user = data_front.loc[data_front["usuario"] == user_name]
        
        text = np.arange(len(data_user[key_x]) ).astype(int).astype(str)
        # ruta emocional 
        fig.add_traces(go.Scatter(x=data_user[key_x], y=data_user[key_y], mode='markers+lines+text', name="max", text=text))
        
        fig.add_shape(type="line", x0=0.5, y0=0, x1=0.5,y1=1, xref='x', yref='y', line=dict(color=color_axes,))
        fig.add_shape(type="line", x0=0, y0=0.5, x1=1,y1=0.5, xref='x', yref='y', line=dict(color=color_axes,))
        fig.add_shape(type="circle", x0=0, y0=0, x1=1, y1=1, line=dict(color=color_axes),)
        fig.update_yaxes(scaleanchor="x", scaleratio=1,)
        fig.update_layout(showlegend=True, title=f"{user_name}")
        fig.update_layout(xaxis=dict(range=[lx, hx], autorange=False), yaxis=dict(range=[ly, hy], autorange=False))



        fig.show()

        

plot_all_roadmap(data_exp,"valence SAM", "arousal SAM",)# users=["AMC"])

Prueba estadística

In [15]:
# y_t = np.array([ data_exp["valence SAM"], data_exp["arousal SAM"]]).transpose(1,0)
# y_p = np.array([ data_exp["valence"], data_exp["arousal"]]).transpose(1,0)

# # prueba t
# print("-- Prueba t-test --")
# rest = stats.ttest_ind(y_t, y_p)
# print(rest)

# rest = stats.ttest_rel(y_t, y_p)
# print(rest)

# # wilcoxon
# print("-- Prueba wilcoxon --")
# r_w = wilcoxon(y_t, y_p)
# print(r_w)